In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import censusdata
import warnings; warnings.simplefilter('ignore')
import math

In [3]:
#censusdata.search('acs5', 2015, 'concept', 'median income')

In [4]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B19301'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19301_001E  | B19301. Per Capita Income in t | Per capita income in the past 12 months (in 2015 Inflati | int  
-------------------------------------------------------------------------------------------------------------------


In [21]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B19013_001E', 'B19301_001E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B19013_001E' : 'Median Household Income', 
                     'B19301_001E' : 'Per Capita Income'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df = df.replace({-666666666.0: None}, regex=True)
df

,TRACT,Median Household Income,Per Capita Income
0,802,60772.0,40709.0
1,10600,89886.0,126401.0
2,40100,152917.0,94733.0
3,40300,140313.0,76141.0
4,50400,67045.0,39287.0
...,...,...,...
199,82100,25125.0,22526.0
200,60200,140592.0,69694.0
201,90400,38750.0,22053.0
202,120500,84850.0,41448.0


In [82]:
df.to_csv('test.csv')

In [22]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

income = pd.DataFrame(0, index=np.arange(len(template.columns[1:])),
                        columns=df.columns[1:])

income.insert(0, 'WARD_PRECI', template.columns[1:])

for i in range(len(income)):
    prct = income['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    tracker = []
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            tracker.append(temp['TRACT'].loc[n])
    med = [float(df[df['TRACT'] == x]['Median Household Income']) for x in tracker if math.isnan(float(df[df['TRACT'] == x]['Median Household Income'])) == False]
    per_c = [float(df[df['TRACT'] == x]['Per Capita Income']) for x in tracker if math.isnan(float(df[df['TRACT'] == x]['Per Capita Income'])) == False]
    try:
        income.loc[i, 'Median Household Income'] = round(sum(med)/len(med))
    except ZeroDivisionError:
        income.loc[i, 'Median Household Income'] = 0
    try:
        income.loc[i, 'Per Capita Income'] = round(sum(per_c)/len(per_c))
    except ZeroDivisionError:
        income.loc[i, 'Per Capita Income'] = 0
        
income.to_csv('Income_Metrics_By_Precinct_2015-2019.csv')
        
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

inc = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=income.columns[1:])
inc.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(inc)):
    neighb = inc['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    track = []
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            track.append(temp['WARD_PRECI'].loc[n])
    med = []
    per_c = []
    try:
        for x in track:
            filt = income['WARD_PRECI'] == x
            if float(income[filt]['Median Household Income']) >= 0:
                med.append(float(income[filt]['Median Household Income']))
            if float(income[filt]['Per Capita Income']) >= 0:
                per_c.append(float(income[filt]['Per Capita Income']))
    except TypeError:
        print(income[filt]['Median Household Income'])
    try:
        inc['Median Household Income'].loc[i] = round(sum(med)/len(med))
        inc['Per Capita Income'].loc[i] = round(sum(per_c)/len(per_c))
    except ZeroDivisionError:
        pass

inc = inc.drop(25)
inc

,NEIGHBORHOOD,Median Household Income,Per Capita Income
0,Roslindale,86274,41576
1,Jamaica Plain,83737,45825
2,Mission Hill,42231,28948
3,Longwood,42215,30482
4,Bay Village,89603,93688
5,Leather District,71872,64531
6,Chinatown,80738,79110
7,North End,109961,90945
8,Roxbury,41549,25872
9,South End,80516,75928


In [17]:
income

,WARD_PRECI,Median Household Income,Per Capita Income
0,0113,61660,32333
1,0102,81150,46397
2,0105,76775,40514
3,0203,119741,73158
4,0205,134370,75792
...,...,...,...
250,1818,80374,35657
251,0502A,49867,39153
252,0115,0,8930
253,0104,70361,41089


In [23]:
inc.to_csv('Boston_Neighborhood_Income_Measures_ACS_Data_2015-19.csv')

In [106]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B19301'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19301_001E  | B19301. Per Capita Income in t | Per capita income in the past 12 months (in 2015 Inflati | int  
-------------------------------------------------------------------------------------------------------------------


In [117]:
censusdata.printtable(censusdata.censustable('acs3', 2012, 'B19013'))
df = censusdata.download('acs1', 2012, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B19301_001E'])
df

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | B19013.  Median Household Inco | Median household income in the past 12 months (in 2012 i |      
-------------------------------------------------------------------------------------------------------------------


ValueError: Unexpected response (URL: https://api.census.gov/data/2012/acs/acs1?get=NAME,B19301_001E&for=tract:*&in=state:25+county:025): error: unknown/unsupported geography heirarchy 